In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# 제스처 목록
max_num_hands = 1
gesture = {1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 
           6: 'six', 7: 'seven', 8: 'eight', 9: 'nine', 10: 'ten'}

rps_gesture = {1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 
               6: 'six', 7: 'seven', 8: 'eight', 9: 'nine', 10: 'ten'}

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=max_num_hands,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# Gesture recognition model
file = np.genfromtxt('C:\\jun_vs\\gesture_train.csv', delimiter=',')
angle = file[:, :-1].astype(np.float32)
label = file[:, -1].astype(np.float32)
knn = cv2.ml.KNearest_create()
knn.train(angle, cv2.ml.ROW_SAMPLE, label)

# 비디오 캡처 시작
cap = cv2.VideoCapture(0)

# 거리 임계값 설정 (데이터셋에 따라 적절히 조정)
threshold = 1000.0

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        continue

    # 이미지 전처리
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    result = hands.process(img)

    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 3))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z]

            # 관절 간 벡터 계산
            v1 = joint[[0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 0, 13, 14, 15, 0, 17, 18, 19], :]
            v2 = joint[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], :]
            v = v2 - v1

            # 벡터 정규화
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # 벡터 간 각도 계산
            angle = np.arccos(np.einsum('nt,nt->n',
                                        v[[0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18], :],
                                        v[[1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15, 17, 18, 19], :]))

            angle = np.degrees(angle)

            # 제스처 추론
            data = np.array([angle], dtype=np.float32)
            ret, results, neighbours, dist = knn.findNearest(data, 3)
            idx = int(results[0][0])

            # 거리 임계값으로 제스처 신뢰도 판단
            if dist[0][0] > threshold:
                cv2.putText(img, text="?", org=(int(res.landmark[0].x * img.shape[1]), 
                            int(res.landmark[0].y * img.shape[0] + 20)), 
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, 
                            color=(255, 255, 255), thickness=2)
            else:
                # 제스처 결과 표시
                if idx in rps_gesture.keys():
                    cv2.putText(img, text=rps_gesture[idx].upper(), 
                                org=(int(res.landmark[0].x * img.shape[1]), 
                                int(res.landmark[0].y * img.shape[0] + 20)), 
                                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, 
                                color=(255, 255, 255), thickness=2)

            # 손가락 관절 연결 그리기
            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

    # 이미지 출력
    cv2.imshow('Game', img)
    
    # 'q'를 누르면 종료
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()